# Setup

In [1]:
from attention_data import AttentionData
import os
import openai
import torch as t
from transformer_lens import HookedTransformer
%pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

# Set API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
assert OPENAI_API_KEY, "OPENAI_API_KEY environment variable is missing from .env"
openai.api_key = OPENAI_API_KEY

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

device = t.device("cuda" if t.cuda.is_available() else "cpu")


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Get a dataset

%pip install datasets > /dev/null
from datasets import load_dataset
dataset = load_dataset("stas/openwebtext-10k", split="train", trust_remote_code=True)


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Get a model

model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
# Make an AttentionData instance

attention_data = AttentionData(
    model=model, 
    text_batch=dataset['text'][:100], # Speed is sensitive to the number of samples
    openai_model="gpt-3.5-turbo-1106", 
    openai_api_key=OPENAI_API_KEY
)

# Usage
(Note: currently the first token attention seems overly high, might be a bug)

In [5]:
# Let's look at L10H7, which was studied closely here: https://arxiv.org/pdf/2310.04625.pdf
layer = 10
head = 7

In [6]:
prompt, description = attention_data.describe_head(layer=layer, head=head, num_samples=20)

Creating new samples for layer 10 head 7


Token indices sequence length is longer than the specified maximum sequence length for this model (5989 > 1024). Running this sequence through the model will result in indexing errors


Making API call to gpt-3.5-turbo-1106...

Based on the attention patterns provided, the attention head seems to pay attent
ion to specific contextually relevant tokens in each generation step. For instan
ce, in the first example, the attention is focused on tokens related to the subj
ect matter, such as "Former" and "Bush" in the context of "Stephen Hadley," indi
cating an emphasis on names and titles. In other examples, attention is directed
 towards indicative phrases like "Germany coach Joachim Low" and specific termin
ology like "Mueller" and "Russia Probe." Additionally, attention is also drawn t
o punctuation marks and conjunctions, potentially reflecting the importance of s
entence structure and coherence in generating text. Overall, the attention head 
appears to prioritize tokens that are semantically or syntactically significant 
for the given context, aligning with the transformer's mechanism of attending to
 relevant information for language generation.


In [19]:
# The "multiple" is the multiple of the average attention pattern value for a row,
# i.e. a multiple of 2 in a row with 10 tokens means the attention score was 0.2

ranked_multiples = attention_data.get_ranked_multiples(
    head=head, 
    layer=layer, 
    num_multiples=10, 
    display=True
)


In [20]:
# Look at the top occurences for a particular token
example_str_token = ranked_multiples[4][0]

ranked_multiples = attention_data.get_ranked_multiples(
    head=head, 
    layer=layer, 
    num_multiples=2,
    str_token=example_str_token,
    display=True
)

In [22]:
# Look at a random grouping of multiples that were larger than average

random_multiples = attention_data.get_random_multiples(
    head=head, 
    layer=layer, 
    num_multiples=15,
    display=True
)